In [1]:
using DataFrames
using Gadfly
using CurveFit

set_default_plot_size(20cm, 12cm)

INFO: Precompiling module FileIO.
INFO: Precompiling module DataFrames.
INFO: Precompiling module ColorTypes.
INFO: Precompiling module Cairo.
INFO: Precompiling module Gadfly.
INFO: Initializing package repository /mnt/juliabox/.julia/v0.6
INFO: Cloning METADATA from https://github.com/JuliaLang/METADATA.jl
INFO: Initializing package repository /mnt/juliabox/.julia/v0.6
INFO: Cloning METADATA from https://github.com/JuliaLang/METADATA.jl

Use "abstract type ApproxFit end" instead.

Use "abstract type LeastSquares<:ApproxFit end" instead.
INFO: Recompiling stale cache file /mnt/juliabox/.julia/lib/v0.6/Polynomials.ji for module Polynomials.

Use "LinearFit{T}(...) where T" instead.

Use "LogFit{T}(...) where T" instead.

Use "PowerFit{T}(...) where T" instead.

Use "ExpFit{T}(...) where T" instead.


In [103]:
df = readtable("CalculationTime.csv")
dfNaive = df[df[:Type] .== 0, :]
dfBetter = df[df[:Type] .== 1, :]
dfBLAS = df[df[:Type] .== 2, :]

,Type,MatrixRows,ClockTime
1,2,70,1249
2,2,140,9991
3,2,210,33749
4,2,280,81702
5,2,350,156507
6,2,420,274518
7,2,490,444566
8,2,560,666857
9,2,630,949501
10,2,700,1275413


In [104]:
dfMeanNaive = by(dfNaive, :MatrixRows, df -> DataFrame(MeanClockTime = mean(df[:ClockTime]), AberrationClockTime = std(df[:ClockTime])))
dfMeanBetter = by(dfBetter, :MatrixRows, df -> DataFrame(MeanClockTime = mean(df[:ClockTime]), AberrationClockTime = std(df[:ClockTime])))
dfMeanBLAS = by(dfBLAS, :MatrixRows, df -> DataFrame(MeanClockTime = mean(df[:ClockTime]), AberrationClockTime = std(df[:ClockTime])))

,MatrixRows,MeanClockTime,AberrationClockTime
1,70,1250.2,8.61265219184995
2,140,10068.9,164.31301429487155
3,210,34031.8,719.6312018434628
4,280,80898.5,1868.6109665857268
5,350,157887.4,2113.701713424421
6,420,273806.2,5075.608092742299
7,490,435272.9,4748.982544831168
8,560,656499.0,15551.637119966215
9,630,933817.9,8626.1084112259
10,700,1.2808432e6,12512.782404849495


In [105]:
# Change type from DataArrays.DataArray{Float64,1} to Array{Float64,1}
# Important to use in curve_fit()
function changeColumnType(df)
    array = zeros(size(df, 1))
    for i=1:10
        array[i] = df[i]
    end
    return array
end

changeColumnType (generic function with 1 method)

In [106]:
x = dfMeanNaive[:MatrixRows]
x = changeColumnType(x)
y = dfMeanNaive[:MeanClockTime]
y = changeColumnType(y)
fitNaive = curve_fit(Poly, x, y, 2)

Poly(437692.1583333297 - 5162.623866883098*x + 13.366906230674063*x^2)

In [107]:
x = dfMeanBetter[:MatrixRows]
x = changeColumnType(x)
y = dfMeanBetter[:MeanClockTime]
y = changeColumnType(y)
fitBetter = curve_fit(Poly, x, y, 2)

Poly(468483.3583333294 - 5532.837663419893*x + 14.302076607915867*x^2)

In [108]:
x = dfMeanBLAS[:MatrixRows]
x = changeColumnType(x)
y = dfMeanBLAS[:MeanClockTime]
y = changeColumnType(y)
fitBLAS = curve_fit(Poly, x, y, 2)

Poly(111848.54999999891 - 1413.4917640692586*x + 4.340224644403208*x^2)

In [111]:
lNaive = layer(dfMeanNaive, x = :MatrixRows, y = :MeanClockTime, ymin = dfMeanNaive[:MeanClockTime]-dfMeanNaive[:AberrationClockTime], ymax = dfMeanNaive[:MeanClockTime] + dfMeanNaive[:AberrationClockTime], Geom.point, Geom.errorbar, Theme(default_color="blue"))
lBetter = layer(dfMeanBetter, x = :MatrixRows, y = :MeanClockTime, ymin = dfMeanBetter[:MeanClockTime]-dfMeanBetter[:AberrationClockTime], ymax = dfMeanBetter[:MeanClockTime] + dfMeanBetter[:AberrationClockTime], Geom.point, Geom.errorbar, Theme(default_color="orange"))
lBLAS = layer(dfMeanBLAS, x = :MatrixRows, y = :MeanClockTime, ymin = dfMeanBLAS[:MeanClockTime]-dfMeanBLAS[:AberrationClockTime], ymax = dfMeanBLAS[:MeanClockTime] + dfMeanBLAS[:AberrationClockTime], Geom.point, Geom.errorbar, Theme(default_color="red"))
lNaiveCurve = layer(x = x, y = fitNaive(x), Geom.line, Theme(default_color="blue"))
lBetterCurve = layer(x = x, y = fitBetter(x), Geom.line, Theme(default_color="orange"))
lBLASCurve = layer(x = x, y = fitBLAS(x), Geom.line, Theme(default_color="red"))

1-element Array{Gadfly.Layer,1}:
 Gadfly.Layer(nothing, Dict{Symbol,Any}(Pair{Symbol,Any}(:y, [34171.2, -971.894, 6419.19, 56344.5, 148804.0, 2.83798e5, 4.61326e5, 6.81388e5, 9.43984e5, 1.24911e6]),Pair{Symbol,Any}(:x, [70.0, 140.0, 210.0, 280.0, 350.0, 420.0, 490.0, 560.0, 630.0, 700.0])), Gadfly.StatisticElement[], Gadfly.Geom.LineGeometry(Gadfly.Stat.Identity(), false, 2, Symbol("")), Gadfly.Theme(RGB{N0f8}(1.0,0.0,0.0), 0.9mm, 0.45mm, 1.8mm, Function[Compose.circle, Gadfly.square, Gadfly.diamond, Gadfly.cross, Gadfly.xcross, Gadfly.utriangle, Gadfly.dtriangle, Gadfly.star1, Gadfly.star2, Gadfly.hexagon, Gadfly.octagon, Gadfly.hline, Gadfly.vline], 0.3mm, :solid, nothing, nothing, 1.0, nothing, 5.0mm, RGB{N0f8}(0.816,0.816,0.878), Measures.Length{:mm,Float64}[0.5mm, 0.5mm], RGB{N0f8}(0.627,0.627,0.627), 0.2mm, "'PT Sans Caption','Helvetica Neue','Helvetica',sans-serif", 2.82222mm, RGB{N0f8}(0.424,0.376,0.42), "'PT Sans','Helvetica Neue','Helvetica',sans-serif", 3.88056mm, RGB{N0f8}(

In [ ]:
plot(lNaive, lBetter, lBLAS, lNaiveCurve, lBetterCurve, lBLASCurve, Guide.title("Blue - Naive\nOrange - Better\nRed - BLAS"))